In [1]:
# Before beginning this notebook you will need to add the following code to the query tool in pgAdmin: 
# ALTER TABLE counties ADD PRIMARY KEY (fips_date); ALTER TABLE hospitals ADD PRIMARY KEY (fips_date);
# Additionally, you will need to enter your password into the connection_string variable in the config.py file. 
# The place holder text is YOURPASSWORDHERE

In [46]:
# Import dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from config import connection_string
import pandas as pd
import psycopg2 as pg
from flask import Flask, jsonify, render_template
from functools import reduce

In [3]:
 # Create engine using the database file
engine = sqlalchemy.create_engine(connection_string)

In [4]:
# Reflect an existing databade into a new model
Base = automap_base()

In [5]:
# Reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
dbConnection = engine.connect();

In [14]:
# Save references to each table
Counties = Base.classes.counties
Hospitals = Base.classes.hospitals
Vaccinations = Base.classes.vaccinations

In [8]:
Base.classes.keys()

['counties', 'hospitals', 'vaccinations']

In [41]:
merged_df = pd.read_sql("SELECT * FROM counties INNER JOIN hospitals ON counties.fips_date = hospitals.fips_date", dbConnection);

In [42]:
merged_df.head()

,index,cases_to_date,deaths_to_date,fips_date,index,fips_code,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,...,total_ped_patients_hospitalized_con_and_sus_covid_7_day_sum,total_ped_patients_hospitalized_con_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_con_and_sus_covid_7_day_sum,staffed_icu_adult_patients_con_covid_7_day_sum,fips_date
0,21,26881,314,10012020-12-25,21,1001,2020-12-25,581,581,462,...,0,0,462,42,42,42,42,27,27,10012020-12-25
1,24,34774,382,10012021-01-15,24,1001,2021-01-15,602,602,462,...,0,0,462,42,42,42,42,39,39,10012021-01-15
2,25,36617,397,10012021-01-22,25,1001,2021-01-22,602,602,462,...,0,0,462,42,42,42,42,41,40,10012021-01-22
3,31,44055,639,10012021-03-05,31,1001,2021-03-05,602,602,462,...,0,0,462,42,42,40,40,22,22,10012021-03-05
4,33,45263,672,10012021-03-19,33,1001,2021-03-19,602,602,462,...,0,0,462,42,42,33,33,9,9,10012021-03-19


In [40]:
#fips_date = merged_df['fips_date']
#merged_df.drop('fips_date', axis=1, inplace=True)
#merged_df['fips_date'] = fips_date
#fips_date
merged_df.drop(columns={merged_df.columns[3]}, inplace=True)
merged_df

,index,cases_to_date,deaths_to_date,index,fips_code,collection_week,total_beds_7_day_sum,all_adult_hospital_beds_7_day_sum,all_adult_hospital_inpatient_beds_7_day_sum,inpatient_beds_used_7_day_sum,...,total_adult_patients_hospitalized_con_covid_7_day_sum,total_ped_patients_hospitalized_con_and_sus_covid_7_day_sum,total_ped_patients_hospitalized_con_covid_7_day_sum,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_con_and_sus_covid_7_day_sum,staffed_icu_adult_patients_con_covid_7_day_sum
0,21,26881,314,21,1001,2020-12-25,581,581,462,445,...,281,0,0,462,42,42,42,42,27,27
1,24,34774,382,24,1001,2021-01-15,602,602,462,423,...,247,0,0,462,42,42,42,42,39,39
2,25,36617,397,25,1001,2021-01-22,602,602,462,360,...,149,0,0,462,42,42,42,42,41,40
3,31,44055,639,31,1001,2021-03-05,602,602,462,315,...,76,0,0,462,42,42,40,40,22,22
4,33,45263,672,33,1001,2021-03-19,602,602,462,284,...,43,0,0,462,42,42,33,33,9,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110337,208898,1738,7,159852,78020,2021-06-18,924,868,728,387,...,11,0,0,784,56,56,45,45,0,0
110338,208902,1772,7,159856,78020,2021-07-16,924,868,728,486,...,69,0,0,784,56,56,35,35,14,14
110339,208911,2242,21,159865,78020,2021-09-17,924,868,728,475,...,7,0,0,784,56,56,54,54,0,0
110340,208912,2258,21,159866,78020,2021-09-24,924,868,728,410,...,20,0,0,784,56,56,29,29,7,7


In [43]:
vax_df = pd.read_sql("SELECT * FROM vaccinations", dbConnection);
hospital_df = pd.read_sql("SELECT * FROM hospitals", dbConnection);
cases_df = pd.read_sql("SELECT * FROM counties", dbConnection);

In [44]:
dfs = [vax_df, hospital_df, cases_df]

In [47]:
df_final = reduce(lambda left,right: pd.merge(left,right,on='fips_date'), dfs)

In [50]:
df_final.head()

,index_x,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_18PlusPop_Pct,...,inpatient_beds_7_day_sum,total_icu_beds_7_day_sum,total_staffed_adult_icu_beds_7_day_sum,icu_beds_used_7_day_sum,staffed_adult_icu_bed_occupancy_7_day_sum,staffed_icu_adult_patients_con_and_sus_covid_7_day_sum,staffed_icu_adult_patients_con_covid_7_day_sum,index,cases_to_date,deaths_to_date
0,49245,38.0,17795,17728,54.4,5292,78.0,97.6,42.6,60.8,...,203,35,35,32,32,20,20,36036,45917,768
1,49246,46.2,55067,52271,57.5,15433,82.9,99.1,52.1,64.7,...,4066,308,308,206,206,71,71,178047,173387,2631
2,49247,39.7,4162,4059,48.3,1756,70.7,99.1,43.4,52.6,...,151,0,0,0,0,0,0,165921,10669,161
3,49248,43.5,13103,12477,51.2,4617,67.9,83.0,50.1,58.9,...,114,12,12,11,11,9,9,188051,33114,787
4,49251,52.8,68276,65043,62.0,21163,87.2,97.9,59.1,69.1,...,3367,728,385,580,322,102,101,163069,150062,2256
